In [1]:
# Import libraries for data manipulation
import pandas as pd
import numpy as np

# Import libraries for data visualization
from matplotlib import pyplot
import matplotlib.pyplot as plt
import seaborn as sns

# Import libraries for building linear regression model
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Ridge

# Import library for preparing data
from sklearn.model_selection import train_test_split

# Import library for data preprocessing

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import mutual_info_regression


# Import libraries for scoring models
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score

#### 1. import, X-y separation, train_test_split
#### 2. EDA
#### 3. create 6 datasets
#### 4. impute, encoding, separate cat/num
#### 5. build models
#### 6. score models

In [2]:
data = pd.read_csv("auto_1993_adj.csv")
df = data.copy()

In [3]:
# df.head()

In [4]:
# df.describe().T

In [5]:
# df.info()

In [6]:
# We will drop "ID" as it is unlikely to contribute to our analysis
features = df.drop(["ID","mpg"], axis=1)
target = df["mpg"]

#### Let's look at the data.

In [7]:
# df.hist(bins=25, figsize = (12,7))

In [8]:
# corr_matrix = df.corr()
# plt.figure(figsize=(8, 6))
# sns.heatmap(corr_matrix, annot=True, cmap="YlOrRd")
# plt.show()

In [9]:
# sns.pairplot(data=df, corner=True, height=3)
# plt.show()


In [10]:
# X = data.copy()
# X = X.dropna(axis=0)
# y = X.pop("mpg")

# # All discrete features should now have integer dtypes (double-check this before using MI!)
# discrete_features = X.dtypes == int

In [11]:
# X = data.copy()
# X = X.dropna(axis=0)
# y = X.pop("mpg")

# # All discrete features should now have integer dtypes (double-check this before using MI!)
# discrete_features = X.dtypes == int

In [12]:
# # print(X.head())
# # X.info()
# y.head()


# def make_mi_scores(X, y, discrete_features):
#     mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
#     print(X.head())
#     mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
#     mi_scores = mi_scores.sort_values(ascending=False)
#     return mi_scores


# X = data.copy()
# X = X.dropna()
# y = X.pop("mpg")
# print(y)
# # print(X.head())
# discrete_features = X.dtypes == int
# mi_scores = make_mi_scores(X, y, discrete_features)
# print(mi_scores)

In [13]:
from sklearn.feature_selection import mutual_info_regression
def make_mi_scores(X, y, discrete_features):
    print(X.head())
    print(y.shape)
    print(discrete_features)
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

mi_scores = make_mi_scores(X, y, discrete_features)
mi_scores[::3]  # show a few features with their MI scores

NameError: name 'X' is not defined

In [15]:
# New features
features["P_to_W"] = features["horsepower"] / features["weight"]
features["D_to_W"] = features["displacement"] / features["weight"]
features["SIZE"] = features["displacement"] + features["weight"]
features["Performance"] = features["horsepower"] + features["acceleration"]

In [16]:
# 6 Distinct Feature Sets
# # Set 1
df1 = features[["displacement","cylinders","horsepower","weight","acceleration","model_year","origin"]]
df2 = features[["displacement", "horsepower","weight","acceleration"]]  
df3 = features[["cylinders","origin"]]                      
df4 = features[["model_year","weight", "cylinders"]]            
df5 = features[["displacement", "horsepower"]]
df6 = features[["Performance"]]      
df6.shape                                        

(398, 1)

In [17]:
def auto_regres(data, model):
    # Separate features and target
    X = data
    global target
    y = target
    # train test split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.25, random_state=16
    )


    # Separate CAT & NUM features
    cat_cols = ["cylinders", "origin"]
    num_cols = ["displacement", "horsepower", "weight", "acceleration", "model_year", "P_to_W","D_to_W","SIZE","Performance"]
    cat_cols = [col for col in data.columns if col in cat_cols]
    num_cols = [col for col in data.columns if col in num_cols]
    # print(cat_cols)
    # print(num_cols)

    # Transform NUM Data
    numerical_transformer = Pipeline(
        steps=[
            ("imputer", SimpleImputer()),
            ("scale", StandardScaler()),
        ]
    )

    # Transform CAT Data
    categorical_transformer = Pipeline(
        steps=[
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("onehot", OneHotEncoder(handle_unknown="ignore")),
        ]
    )
    
    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numerical_transformer, num_cols),
            ("cat", categorical_transformer, cat_cols),
        ]
    )

    # # Define models
    # knn = KNeighborsRegressor()
    # svr = SVR()
    # ridge = Ridge()

    # Preprocess data then create model
    mod_pipeline = Pipeline(steps=[("preprocessor", preprocessor), ("model", model)])
    # print(X_train.tail(5))
    # print(X_train.shape)
    # print(type(X_train))
    # print(y_train.tail(5))
    # print(y_train.shape)
    # print(type(y_train))

    # Preprocess training data, fit model
    mod_pipeline.fit(X_train, y_train)

    # Get predictions for training data
    preds = mod_pipeline.predict(X_train)

    return " MAE:", mean_squared_error(y_train, preds), "R_2:", r2_score(y_train, preds)

In [18]:
# Define models
knn = KNeighborsRegressor()
svr = SVR()
ridge = Ridge()

In [20]:
print("df1",auto_regres(df1, knn))
print("df2",auto_regres(df2, knn))
print("df3",auto_regres(df3, knn))
print("df4",auto_regres(df4, knn))
print("df5",auto_regres(df5, knn))
print("df6",auto_regres(df6, knn))


df1 (' MAE:', 5.870642953020134, 'R_2:', 0.8990686907003489)
df2 (' MAE:', 10.70973154362416, 'R_2:', 0.8158724290344264)
df3 (' MAE:', 21.628995973154364, 'R_2:', 0.6281424539224804)
df4 (' MAE:', 5.46551543624161, 'R_2:', 0.9060338648778616)
df5 (' MAE:', 10.899906040268457, 'R_2:', 0.8126028449198216)
df6 (' MAE:', 14.597138255033558, 'R_2:', 0.7490380035204454)


In [ ]:
# def cv_regres(data, model):
#     # Separate features and target
#     X = data.drop("mpg", axis=1)
#     y = data["mpg"]

#     # train test split
#     X_train, X_test, y_train, y_test = train_test_split(
#         X, y, test_size=0.25, random_state=16
#     )

#     # Separate CAT & NUM features
#     cat_cols = ["cylinders", "origin"]
#     num_cols = ["displacement", "horsepower", "weight", "acceleration", "model_year"]
#     cat_cols = [col for col in data.columns if col in cat_cols]
#     num_cols = [col for col in data.columns if col in num_cols]

#     # Transform NUM Data
#     numerical_transformer = Pipeline(
#         steps=[
#             ("imputer", SimpleImputer()),
#             ("scale", StandardScaler()),
#         ]
#     )

#     # Transform CAT Data
#     categorical_transformer = Pipeline(
#         steps=[
#             ("imputer", SimpleImputer(strategy="most_frequent")),
#             ("onehot", OneHotEncoder(handle_unknown="ignore")),
#         ]
#     )
#     # Create Data Preprocessor Process
#     preprocessor = ColumnTransformer(
#         transformers=[
#             ("num", numerical_transformer, num_cols),
#             ("cat", categorical_transformer, cat_cols),
#         ]
#     )

#     # Define models
#     knn = KNeighborsRegressor()
#     svr = SVR()
#     ridge = Ridge()

#     # Preprocess data then create model
#     mod_pipeline = Pipeline(steps=[("preprocessor", preprocessor), ("model", model)])

#     # Cross-Validation Code
#     scores = -1 * cross_val_score(
#         mod_pipeline, X, y, cv=5, scoring="neg_mean_absolute_error"
#     )

#     print("MAE scores:\n", scores)

In [ ]:
auto_regres(df1,ridge)

In [ ]:
# simple_imputer = SimpleImputer()
# X_train_imputed = pd.DataFrame(simple_imputer.fit_transform(X_train))
# X_test_imputed = pd.DataFrame(simple_imputer.transform(X_test))
# X_train_imputed.columns = X_train.columns
# X_test_imputed.columns = X_test.columns

In [ ]:
# Preprocessing for numerical data
numerical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer()),
        ("scale", StandardScaler()),
    ]
)

# Preprocessing for categorical data
categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
    ]
)
# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_transformer, num_cols),
        ("cat", categorical_transformer, cat_cols),
    ]
)

In [ ]:
# # Apply ordinal encoder to each column with categorical data
# ordinal_encoder = OrdinalEncoder()
# label_X_train[object_cols] = ordinal_encoder.fit_transform(X_train[object_cols])
# label_X_valid[object_cols] = ordinal_encoder.transform(X_valid[object_cols])

In [ ]:
# # One Hot Encoder
# # Apply one-hot encoder to each column with categorical data
# OHE = OneHotEncoder(handle_unknown="ignore", sparse=False)
# OH_cols_train = pd.DataFrame(OHE.fit_transform(X_train_imputed[cat_cols]))
# OH_cols_valid = pd.DataFrame(OHE.transform(X_test_imputed[cat_cols]))

# # One-hot encoding removed index; put it back
# OH_cols_train.index = X_train_imputed.index
# OH_cols_valid.index = X_test_imputed.index

# # Remove categorical columns (will replace with one-hot encoding)
# num_X_train_imputed = X_train_imputed.drop(cat_cols, axis=1)
# num_X_test_imputed = X_test_imputed.drop(cat_cols, axis=1)

# # Add one-hot encoded columns to numerical features
# OH_X_train_imputed = pd.concat([num_X_train_imputed, OH_cols_train], axis=1)
# OH_X_test_imputed = pd.concat([num_X_test_imputed, OH_cols_valid], axis=1)

# # Ensure all columns have string type
# OH_X_train_imputed.columns = OH_X_train_imputed.columns.astype(str)
# OH_X_test_imputed.columns = OH_X_test_imputed.columns.astype(str)

# # print("MAE from Approach 3 (One-Hot Encoding):")
# # print(score_dataset(OH_X_train_imputed, OH_X_test_imputed, y_train, y_valid))

In [ ]:
# OH_X_train_imputed.head(15)
# OH_X_train_imputed.rename(
#     columns={
#         "0": "cyl_3",
#         "1": "cyl_4",
#         "2": "cyl_5",
#         "3": "cyl_6",
#         "4": "cyl_8",
#         "5": "org_1",
#         "6": "org_2",
#         "7": "org_3"},
#     inplace=True,
# )

#### Modeling

In [ ]:
# Define models
knn = KNeighborsRegressor()
svr = SVR()
ridge = Ridge()

# Bundle preprocessing and modeling code in a pipeline
clf = Pipeline(steps=[("preprocessor", preprocessor), ("model", knn)])

# Preprocessing of training data, fit model
clf.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = clf.predict(X_train)

# score = mean_absolute_error(y_valid, preds)
print(" MAE:", mean_squared_error(y_train, preds))
print("R_2:", r2_score(y_train, preds))